In [185]:
# selenium 라이브러리에서 webdriver 모듈 임포트, 웹 브라우저를 자동으로 제어하기 위한 기능을 제공
from selenium import webdriver
# 크롬 웹드라이버를 위한 서비스 객체를 사용하기 위해 Service 모듈 임포트
from selenium.webdriver.chrome.service import Service
# 크롬 드라이버의 자동 업데이트를 위해 webdriver_manager에서 ChromeDriverManager를 임포트
from webdriver_manager.chrome import ChromeDriverManager
# 크롬 옵션을 설정하기 위한 Options 모듈을 임포트 일반적으로 브라우저의 특정 옵션을 설정할 때 사용
from selenium.webdriver.chrome.options import Options
# html요소 접근
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import pandas as pd
from bs4 import BeautifulSoup
import pyautogui
import time

# 크롬 브라우저 꺼짐 방지
chrome_options = Options()
chrome_options.add_experimental_option("detach",True)
# 불필요한 에레 메세지 제거
chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])



In [186]:
# ChromeDriverManager를 사용해 버전에 맞는 웹드라이버를 다운로드하여 해당 경로를 셀레니움에 전달
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
# 웹페이지가 로딩될대까지 5초대기
driver.implicitly_wait(5)
# 브라우저 크기 최대화
driver.maximize_window()

In [187]:
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
driver.get(url)

In [188]:
# 첫회차 선택
start = driver.find_element(By.CSS_SELECTOR,'#drwNoStart')
start.click()
time.sleep(1)
pyautogui.press('END') #ENd누름
time.sleep(1)
start.click()


In [189]:
# 마지막회차 선택
start = driver.find_element(By.CSS_SELECTOR,'#drwNoEnd')
start.click()
time.sleep(1)
pyautogui.press('HOME') #ENd누름
time.sleep(1)
start.click()

In [190]:
# 다운로드
downloadBtn = driver.find_element(By.CSS_SELECTOR,'#exelBtn')
downloadBtn.click()
time.sleep(2)

In [191]:
# 엑셀 파일 경로
xls_file_path = 'C:\\Users\\kh502\\Downloads\\excel.xls'

In [192]:
with open(xls_file_path,'r',encoding='EUC-KR') as file:
    html_content = file.read()

In [193]:
soup  = BeautifulSoup(html_content,'html.parser')

In [194]:
# str(soup) DOM객체를 순수 HTML문자열로 변환
html_str = str(soup)
# pd.read_html : html문자열 중에 모든 table 요소를 찾아 데이터 프레임으로 가져와 리스트에 넣어준다.  
tables = pd.read_html(str(soup))

In [195]:
df = tables[1].iloc[2:]
df.columns = tables[1].iloc[1]
df.to_excel('D:\\kdt\\pythonstudy\\lotto\\lotto.xlsx', index=False)

In [196]:
df.shape

(1113, 20)

In [197]:
# 현재 포커스된 창만 닫기
# driver.close()
# 웹드라이버와 함께 연결된 모든 창을 닫고 셀레니엄 세션을 완전히 종료!
driver.quit()

In [198]:
df = pd.read_excel('D:\\kdt\\pythonstudy\\lotto\\lotto.xlsx')


In [199]:
df.head(1)

,년도,회차,추첨일,당첨자수,당첨금액,당첨자수.1,당첨금액.1,당첨자수.2,당첨금액.2,당첨자수.3,당첨금액.3,당첨자수.4,당첨금액.4,1,2,3,4,5,6,보너스
0,2024,1113,2024.03.30,14,"1,987,426,822원",57,"81,356,654원",2936,"1,579,472원",146997,"50,000원",2444535,"5,000원",11,13,20,21,32,44,8


In [200]:
# 과거 당첨 로또 번호 6자리 목록만 추출
past_lotto_number_list = df.iloc[:,-7:-1]

In [201]:
# 각 열의 가중치 계산 
weights = [ past_lotto_number_list[str(i)].value_counts(normalize=True) for i in range(1,6+1) ]
# weights

In [202]:
def generate_numbers(past_lotto_number_list, weights) :
    dupCnt = 0
    
    while True:
        # 가중치를 반영한 로또 번호생성
        numbers = [ np.random.choice(i.index, p=i.values)  for i in weights]

        # 과거 당첨번호와 중복되는지 체크
        if not any((past_lotto_number_list == numbers).all(axis=1)):
            return numbers
        else:
            print(f'중복{++dupCnt}')
    

In [203]:
# 사용자에개 로또번호 추출 횟수 입력받기
cnt = int(input('로또 번호 몇개를 생성할까요?'))

로또 번호 몇개를 생성할까요?5


In [204]:
for i in range(cnt) :
    number = generate_numbers(past_lotto_number_list, weights)
    print(f'{i+1}번째 추출된 번호 : {number}')

1번째 추출된 번호 : [2, 23, 14, 33, 30, 44]
2번째 추출된 번호 : [10, 14, 29, 33, 24, 45]
3번째 추출된 번호 : [4, 14, 29, 11, 21, 37]
4번째 추출된 번호 : [3, 25, 26, 17, 30, 39]
5번째 추출된 번호 : [12, 11, 11, 37, 22, 44]


In [205]:
past_lotto_number_list.iloc[0]

1    11
2    13
3    20
4    21
5    32
6    44
Name: 0, dtype: int64

In [206]:
past_lotto_number_list

,1,2,3,4,5,6
0,11,13,20,21,32,44
1,16,20,26,36,42,44
2,3,13,30,33,43,45
3,3,7,11,20,22,41
4,10,12,13,19,33,40
...,...,...,...,...,...,...
1108,16,24,29,40,41,42
1109,14,27,30,31,40,42
1110,11,16,19,21,27,31
1111,9,13,21,25,32,42


In [207]:
(past_lotto_number_list == [11,13,20,21,32,44]).all(axis=1)

0        True
1       False
2       False
3       False
4       False
        ...  
1108    False
1109    False
1110    False
1111    False
1112    False
Length: 1113, dtype: bool